## Accessing and downloading Ag1000G phase 3 dataset via Python API
### Tin-Yu Hui, 31/03/2022 (updated 04/05/2022)
The main goal for me is to use the minimum python skills to select the required mosquito samples and sites (SNPs) from the huge Ag1000G phase 3 dataset, save and export them so I can perform my own analyses in R. I should mention that Alistair Miles' brilliant python package "scikit-allel" contains lots of useful tools for population genetic analysis. If you are not as stubborn as me then you should make good use of those tools, in python. 

The terminologies and explanations here may be inaccurate, please always consult the official user guide:  
https://malariagen.github.io/vector-data/landing-page.html. 
Besides, this document reflects only my personal experience - it does not illustrate the best practice. Some comments below are fueled by my inability and frustration towards python and they should never be taken seriously. I am happy to share this notebook with everyone in the workgroup, although it is extremely likely that I am the only audience. 

Specials thanks to Jon Brenas for his contribution and patience. 

#### 0. Install API
The first step is to install the malariagen_data API, and you only need to do it once per PC. If you're using Anaconda (on Windows OS) like me you can run this line on Anaconda Prompt (command line window) as an Adminstrator. Or if you're using Google Colab (cloud computing) then you have to install every time. 

In [4]:
# OR RUN pip install malariagen_data on anaconda command prompt
# RUN ONLY ONCE
!pip install -q malariagen_data

#### 1. Import API
After the initial installation you can use the API by importing it into your current python session. If everything is done correctly you will see the some basic info as shown below. 

In [1]:
# IMPORT DATA
import malariagen_data
ag3=malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 data resource API>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Cohorts analysis        : 20211101
Species analysis        : aim_20200422
Site filters analysis   : dt_20200416
Software version        : 3.0.0
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see: 
https://malariagen.github.io/vector-data/ag3/api.html

We also need to import some other python packages. 

In [6]:
# IMPORT SOME TOOLS
import pandas
import numpy as np
import allel
import dask.array as da
from itertools import compress

Then I am going to walk through some important datasets (and also functions, or methods) within it. 

##### 1.1 sample_metadata (about mosquito individuals)
Within ag3, ag3.sample_metadata() shows metadata about mosquito individuals. We can use it to view or select inviduals by country, location, year, or by species. 

In [2]:
# METADATA (NO FILTERING, SHOW ALL 3081 SAMPLES)
metadata=ag3.sample_metadata()
metadata

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
0,AR0047-C,LUA047,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,...,coluzzii,AGO,Luanda,AO-LUA,Luanda,coluzzii,AO-LUA_colu_2009,AO-LUA_colu_2009_04,AO-LUA_Luanda_colu_2009,AO-LUA_Luanda_colu_2009_04
1,AR0049-C,LUA049,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,...,coluzzii,AGO,Luanda,AO-LUA,Luanda,coluzzii,AO-LUA_colu_2009,AO-LUA_colu_2009_04,AO-LUA_Luanda_colu_2009,AO-LUA_Luanda_colu_2009_04
2,AR0051-C,LUA051,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,...,coluzzii,AGO,Luanda,AO-LUA,Luanda,coluzzii,AO-LUA_colu_2009,AO-LUA_colu_2009_04,AO-LUA_Luanda_colu_2009,AO-LUA_Luanda_colu_2009_04
3,AR0061-C,LUA061,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,...,coluzzii,AGO,Luanda,AO-LUA,Luanda,coluzzii,AO-LUA_colu_2009,AO-LUA_colu_2009_04,AO-LUA_Luanda_colu_2009,AO-LUA_Luanda_colu_2009_04
4,AR0078-C,LUA078,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,...,coluzzii,AGO,Luanda,AO-LUA,Luanda,coluzzii,AO-LUA_colu_2009,AO-LUA_colu_2009_04,AO-LUA_Luanda_colu_2009,AO-LUA_Luanda_colu_2009_04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,AD0494-C,80-2-o-16,Martin Donnelly,Lab Cross,LSTM,-1,-1,53.409,-2.969,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,AD0495-C,80-2-o-17,Martin Donnelly,Lab Cross,LSTM,-1,-1,53.409,-2.969,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3078,AD0496-C,80-2-o-18,Martin Donnelly,Lab Cross,LSTM,-1,-1,53.409,-2.969,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079,AD0497-C,80-2-o-19,Martin Donnelly,Lab Cross,LSTM,-1,-1,53.409,-2.969,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# SHOW METADATA COLUMN NAMES
metadata.columns

Index(['sample_id', 'partner_sample_id', 'contributor', 'country', 'location',
       'year', 'month', 'latitude', 'longitude', 'sex_call', 'sample_set',
       'release', 'aim_species_fraction_colu', 'aim_species_fraction_arab',
       'aim_species_gambcolu_arabiensis', 'aim_species_gambiae_coluzzii',
       'aim_species', 'country_iso', 'admin1_name', 'admin1_iso',
       'admin2_name', 'taxon', 'cohort_admin1_year', 'cohort_admin1_month',
       'cohort_admin2_year', 'cohort_admin2_month'],
      dtype='object')

In [4]:
# FOR EXAMPLE, SEE ONLY BURKINA FASO SAMPLES
metadata[metadata["country"]=="Burkina Faso"]

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
81,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
82,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
83,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
84,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
85,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,AB0314-C,6775,Nora Besansky,Burkina Faso,Monomtenga,2004,8,12.060,-1.170,F,...,gambiae,BFA,Centre-Sud,BF-07,Bazega,gambiae,BF-07_gamb_2004,BF-07_gamb_2004_08,BF-07_Bazega_gamb_2004,BF-07_Bazega_gamb_2004_08
373,AB0315-C,6777,Nora Besansky,Burkina Faso,Monomtenga,2004,8,12.060,-1.170,F,...,gambiae,BFA,Centre-Sud,BF-07,Bazega,gambiae,BF-07_gamb_2004,BF-07_gamb_2004_08,BF-07_Bazega_gamb_2004,BF-07_Bazega_gamb_2004_08
374,AB0316-C,6779,Nora Besansky,Burkina Faso,Monomtenga,2004,8,12.060,-1.170,F,...,gambiae,BFA,Centre-Sud,BF-07,Bazega,gambiae,BF-07_gamb_2004,BF-07_gamb_2004_08,BF-07_Bazega_gamb_2004,BF-07_Bazega_gamb_2004_08
375,AB0318-C,5072,Nora Besansky,Burkina Faso,Monomtenga,2004,7,12.060,-1.170,F,...,gambiae,BFA,Centre-Sud,BF-07,Bazega,gambiae,BF-07_gamb_2004,BF-07_gamb_2004_07,BF-07_Bazega_gamb_2004,BF-07_Bazega_gamb_2004_07


In [5]:
# GET ONLY BURKINA FASO, 2012, 2014 (TWO LOGICAL STATEMENTS)
BF_metadata=metadata[(metadata["country"]=="Burkina Faso") & (metadata["year"].isin([2012, 2014]))]
BF_metadata

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
81,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
82,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
83,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
84,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
85,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,AB0533-C,BF13-18,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
360,AB0536-C,BF13-31,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
361,AB0537-C,BF13-32,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
362,AB0538-C,BF13-33,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07


You can also select individuals via the pre-compiled sample_sets to select individuals. You may loosely regard them as different populations, although you should always double check with the metadata. There is a performance advantage for using sample_sets (if applicable) as the genotype dask arrays have been optimised towards this. 

For example, mosquitoes from Burkina Faso 2012-2014 are in sample_sets "AG1000G-BF-A", "AG1000G-BF-B". Note that there are 3 arabiensis which need to be removed, but I guess I can do it in R. 

In [2]:
# ANOTHER WAY TO SUBSET INDIVIDUALS FROM METADATA IS VIA sample_set
BF_metadata=ag3.sample_metadata(sample_sets=["AG1000G-BF-A", "AG1000G-BF-B"])
BF_metadata

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
0,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
1,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
2,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
3,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
4,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,AB0533-C,BF13-18,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
279,AB0536-C,BF13-31,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
280,AB0537-C,BF13-32,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07
281,AB0538-C,BF13-33,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.238,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2014,BF-09_gamb_2014_07,BF-09_Houet_gamb_2014,BF-09_Houet_gamb_2014_07


In [4]:
# SEE sample_ID OF THOSE IN sample_sets BFA AND BFA
# TURN INTO LIST IF YOU WANT TO DO SOMETHING
BF_metadata["sample_id"]
#list(BF_metadata["sample_id"])

0      AB0085-Cx
1      AB0086-Cx
2       AB0087-C
3       AB0088-C
4      AB0089-Cx
         ...    
278     AB0533-C
279     AB0536-C
280     AB0537-C
281     AB0538-C
282     AB0408-C
Name: sample_id, Length: 283, dtype: object

Obviously I would like to save a copy of the metadata of all the 3000+ samples as a .csv file.  

In [9]:
# WRITE METADATA TO CSV
metadata.to_csv('metadata.csv', index=False)

##### 1.2 snp_calls (genotypes, positions, ref and alt alleles)
ag3.snp_calls() stores genotypes, physical positions, ref and alt alleles and many more. You may think ds_snps is a dataset of datasets. It is huge, and the contents are mostly in dasks arrays format. 

When using snp_calls you can perform some filtering (or compressing, see later paragraph) to take subsets of samples and sites. Here I am choosing chromosome 3R (add a colon : after 3R if you want to choose positions as well, not tried by myself) from the two relevant Burkina Faso sample_sets. I am also picking SNPs which had passed for one of the three quality filters by specifying site_mask=. The filter I use here is "gamb_colu", which works best when I am analysing coluzzii and gambiae only. 

In [3]:
# WHEN WE MOVE ONTO GENOTYPES WE MUST CREATE SNP CALL
# APPARENTLY I CAN FILTER FOR QUALITY HERE AS WELL VIA site_mask
# AND ALSO TO SELECT BURKINA FASO SAMPLES
ds_snps=ag3.snp_calls(region="3R", 
                      sample_sets=["AG1000G-BF-A", "AG1000G-BF-B"], 
                      site_mask="gamb_colu")
ds_snps
# DO I NEED TO WORRY ABOUT THE WARNING?

C:\Users\Yue\anaconda3\lib\site-packages\dask\array\core.py:1438: RuntimeWarning: overflow encountered in long_scalars
  return reduce(mul, self.shape, 1)
C:\Users\Yue\anaconda3\lib\site-packages\dask\array\core.py:1443: RuntimeWarning: overflow encountered in long_scalars
  return self.size * self.dtype.itemsize


<xarray.Dataset>
Dimensions:                             (variants: 37199402, alleles: 4,
                                         samples: 283, ploidy: 2)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(468914,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(468914,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(181,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(468914, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(263710,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(263710,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(263710,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(263710, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(263710, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(263710, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(263710, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(263710, 50, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [5]:
# IF site_mask IS SPECIFIED IN THE PREVIOUS STEP THEN YOU SHOULD GET ALL PASSED STIES
#filter_pass=ds_snps["variant_filter_pass_gamb_colu"].data
#filter_pass

dask.array<getitem, shape=(37199402,), dtype=bool, chunksize=(279737,), chunktype=numpy.ndarray>

In [26]:
# COUNT HOW MANY SITES THAT HAVE PASSED, SHOULD EQUAL TOTAL NUMBER OF SITES
#sum(filter_pass.compute())

37199402

In ds_snps["call_genotype"] you find the genotype, and ds_snps["variant_position"] you find the physical position (base pairs). Always pay attention to the dimensions. The genotype is a 3D array: the first dimension is the SNPs, then individuals, and the third dimension is ploidy (diploid, hence 2). 

Note: there are 37.199 million sites on chromosome 3R that had passed our filter. 

In [4]:
# SEE THE GENOTYPES, AS A DASK ARRAY (?)
genotype=ds_snps["call_genotype"]
genotype

<xarray.DataArray 'call_genotype' (variants: 37199402, samples: 283, ploidy: 2)>
dask.array<getitem_variadic, shape=(37199402, 283, 2), dtype=int8, chunksize=(279737, 50, 2), chunktype=numpy.ndarray>
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(468914,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(468914,), meta=np.ndarray>
    sample_id         (samples) <U24 dask.array<chunksize=(181,), meta=np.ndarray>
Dimensions without coordinates: variants, samples, ploidy

In [5]:
# ACCESS THE GENOTYPE ELEMENTS
# FIRST SITE ACROSS ALL 283 INDIVIDUALS
#genotype[0].data.compute()

In [5]:
# FIRST 10 STIES FOR THE SECOND (1) INDIVIDUAL
# 0:10 HAS 10 ELEMENTS... STUPID...
genotype[0:10, 1].data.compute()

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int8)

In [5]:
# SEE PHYSICAL POSITION
POS=ds_snps["variant_position"]
POS

<xarray.DataArray 'variant_position' (variants: 37199402)>
dask.array<getitem, shape=(37199402,), dtype=int32, chunksize=(487437,), chunktype=numpy.ndarray>
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(468914,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(468914,), meta=np.ndarray>
Dimensions without coordinates: variants

In [7]:
# ACCESS POS AS A NUMERIC VECTOR
# WHY 0:10 ONLY GIVES 10 NUMBERS? STUPID!
POS[0:10].data.compute()

array([180, 185, 236, 239, 240, 241, 242, 244, 245, 246])

In [8]:
# SMALLEST POS. USING DASK ARRAY FUNCTIONS
da.min(POS.data).compute()

180

In [9]:
# LARGEST POS. OBVIOUSLY WE DON'T HAVE ALL THE SITES. 
da.max(POS.data).compute()

53196522

##### 1.3 geneset information (SNPeff, which sites are intergenic?)
ag3.geneset() gives the starting and ending positions (base pairs) of genes and other things. It is similar to SNPeff in phase 2 release, I believe. It tells us where the intergenic sites are. 

In [6]:
# SEE WHAT'S INSIDE
ag3.geneset()
#ag3.geneset().type.value_counts()

,contig,source,type,start,end,score,strand,phase,ID,Parent,Name,description
0,2L,VectorBase,chromosome,1,49364325,NaN,NaN,NaN,2L,NaN,NaN,NaN
1,2L,VectorBase,gene,157348,186936,NaN,-,NaN,AGAP004677,NaN,NaN,methylenetetrahydrofolate dehydrogenase(NAD ) ...
2,2L,VectorBase,mRNA,157348,181305,NaN,-,NaN,AGAP004677-RA,AGAP004677,NaN,NaN
3,2L,VectorBase,three_prime_UTR,157348,157495,NaN,-,NaN,NaN,AGAP004677-RA,NaN,NaN
4,2L,VectorBase,exon,157348,157623,NaN,-,NaN,NaN,AGAP004677-RA,AGAP004677-RB-E4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
196140,Y_unplaced,VectorBase,five_prime_UTR,47932,48111,NaN,+,NaN,NaN,AGAP029375-RA,NaN,NaN
196141,Y_unplaced,VectorBase,exon,47932,48138,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E2,NaN
196142,Y_unplaced,VectorBase,CDS,48112,48138,NaN,+,0.0,AGAP029375-PA,AGAP029375-RA,NaN,NaN
196143,Y_unplaced,VectorBase,exon,48301,48385,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E3,NaN


In [7]:
# FOR EXAMPLE I AM CREATING A TABLE WITH ALL THE GENE STARTING AND ENDING POINTS
# FOR 3R. THERE ARE 2686 GENES
ag3.geneset().query("type == 'gene' and contig == '3R'")[['start', 'end']]

,start,end
132569,13603,22079
132581,24056,39486
132608,39643,41560
132617,57849,62977
132637,82151,83064
...,...,...
173838,53015041,53016827
173852,53071753,53087640
173870,53091293,53094028
173886,53112380,53112981


#### 2. Subset individuals and/or SNPs
Now we have located all the necessary datasets, and it's time to use them to pick the sites required. It is a bit challenging as there are multiple datasets and logical statements. 

We need to find three things: biallelic (including fixed) sites, intergenic sites, and sites with complete information (no missing values). 

##### 2.1 Find all biallelic and fixed sites (updated 03/05/2022)
Here we need to find biallelic and fixed sites. Finding fixed sites allows us to calculate nucleotide diversity. From the genotype dataset we produce a count table for the four alleles per site. Find the sites with one or two distinct types of alleles. Then create a boolean (binary, True/False) vector, with length 37.199M (all passed sites) for 3R to store the results. 

In [7]:
# IT SEEMS THAT I NEED TO REWRITE THE genotype DEFINED ABOVE
genotype=allel.GenotypeDaskArray(ds_snps['call_genotype'])
genotype

C:\Users\Yue\anaconda3\lib\site-packages\dask\array\core.py:3095: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<GenotypeDaskArray shape=(37199402, 283, 2) dtype=int8>

In [8]:
# CALCULATE ALLELE COUNT
# MY VERSION REQUIRES ME TO SPECIFY max_allele (MAX ALTERNATE ALLELE=3)
# RETURN A TABLE WITH FOUR COLUMNS (REFERENCE ALLELE + 3 ALTERNATE ALLELES)
allele_count=genotype.count_alleles(max_allele=3)
allele_count

<AlleleCountsDaskArray shape=(37199402, 4) dtype=int32>

Previously we used is_biallelic() to find the sites with exactly two types of alleles. This excluded fixed sites with no variation. If we want to include fixed sites we need to use allelism() to calculate the number of distinct alleles per site, from which we select the sites. 

The codes using is_biallelic() are still shown below for comparison, but we're not using the results. 

In [10]:
# FIND BIALLELIC. RETURN A VECTOR WITH TRUE/FALSE
biallelic=allele_count.is_biallelic()
biallelic[0:10].compute()

array([False,  True, False, False, False,  True, False, False, False,
        True])

In [17]:
# HOW MANY BIALLELIC SNPS ARE THERE (OUT OF 37.199M)?
da.sum(biallelic).compute()

10774372

In [9]:
# COUNT HOW MANY ALLELES PER SITE. 1=FIXED, 2=BIALLELIC, 3,4=MULTIALLELIC
allelism=allele_count.allelism()
allelism[0:10].compute()

array([1, 2, 1, 1, 1, 2, 1, 1, 1, 2])

In [10]:
# A BOOLEAN VECTOR INDICATING FIXED OR BIALLELIC SITES
fixed_biallelic=(allelism<=2)
fixed_biallelic[0:10].compute()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [41]:
da.sum(fixed_biallelic).compute()

34983100

There are 10.7M biallelic SNPs, but when we include the fixed sites as well the number increases to 34.9M. The boolean vector fix_bialellic is what I need. 

In production, I may want to find biallelic loci across all the populations, not only within a subset of population(s). Or within a speices. 

I just notice that I cannot spell the word "biallelic" properly. 

##### 2.2 Find intergenic SNPs
From geneset we have the starting and ending positions of all the genes. Intergenic sites are those locate after the end of the previous gene but before the start of the next gene. Similar to finding biallelic SNPs we need to create a boolean vector telling whether the sites are within or outside (outside is True) any gene. I don't claim I understand the complicated code from Jon below. 

In [11]:
# EXTRACT THE START AND END POITNS AS TWO VECTORS
gene_starts, gene_ends = [s for (s,t) in ag3.geneset().query("type == 'gene' and contig == '3R'")[['start', 'end']].itertuples(index=False)], [t for (s,t) in ag3.geneset().query("type == 'gene' and contig == '3R'")[['start', 'end']].itertuples(index=False)]
len(gene_starts)

2686

In [12]:
# SEE WHERE (BASE PAIRS) A GENE BEGINS AND ENDS
gene_starts[0:10]
#gene_ends[0:10]

[13603, 24056, 39643, 57849, 82151, 82151, 149476, 166770, 169278, 172973]

In [13]:
# MAKE SURE I HAVE THE POSITION VECTOR (IF I HAVEN'T ALREADY DONE SO)
# IN MEMORY?
POS=allel.SortedIndex(ds_snps['variant_position'])
POS

0,1,2,3,4,...,37199397,37199398,37199399,37199400,37199401
180,185,236,239,240,...,53196499,53196500,53196502,53196504,53196522


In [14]:
# ????? I'M TOTAL LOST HERE ?????
good_slice = np.zeros(len(gene_starts))
for i in range(0,len(gene_starts)):
  try:
    POS.locate_range(gene_starts[i], gene_ends[i])
    good_slice[i] = True
  except TypeError:
    print('TypeError at gene: ' + str(i))
  except KeyError:
    print('KeyError at gene: ' + str(i))

KeyError at gene: 228
KeyError at gene: 229
KeyError at gene: 429
KeyError at gene: 440
KeyError at gene: 568
KeyError at gene: 569
KeyError at gene: 645
KeyError at gene: 658
KeyError at gene: 660
KeyError at gene: 770
KeyError at gene: 771
KeyError at gene: 793
KeyError at gene: 794
KeyError at gene: 800
KeyError at gene: 803
KeyError at gene: 804
KeyError at gene: 805
KeyError at gene: 822
KeyError at gene: 823
KeyError at gene: 824
KeyError at gene: 825
KeyError at gene: 828
KeyError at gene: 1156
KeyError at gene: 1157
KeyError at gene: 1319
KeyError at gene: 1385
KeyError at gene: 1593
KeyError at gene: 1629
KeyError at gene: 1699
KeyError at gene: 1723
KeyError at gene: 1724
KeyError at gene: 1725
KeyError at gene: 1728
KeyError at gene: 1729
KeyError at gene: 1730
KeyError at gene: 1731
KeyError at gene: 1732
KeyError at gene: 1735
KeyError at gene: 1736
KeyError at gene: 1747
KeyError at gene: 1764
KeyError at gene: 1767
KeyError at gene: 1773
KeyError at gene: 1776
KeyError a

In [15]:
# ?????
len(gene_starts) - np.sum(good_slice)

84.0

In [16]:
# ?????
# SO THIS compress ISN'T THE SAME AS THAT compress?????
sel_gene_starts = list(compress(gene_starts,good_slice))
sel_gene_ends = list(compress(gene_ends,good_slice))

In [17]:
# ?????
# THE TILDE MEANS NEGATION, I GUESS?
intergenic=~POS.locate_ranges(sel_gene_starts, sel_gene_ends)
intergenic[0:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [28]:
# NUMBER OF INTERGENIC SITES (OUT OF 37.199M)
da.sum(intergenic).compute()

18742153

Luckily there aren't too many intergenic sites (18.7M). 

##### 2.4 Sites with no missing alleles
The count_missing() function in scikit-allel returns the number of missing alleles. To find the missing allele count per site we need to specify axis=1. Sites are considered complete if they have zero missing count. 

Note: According to Jon we are summing over (collapsing) all the individuals per sites hence it should be performed on axis=1. 

In [18]:
# COUNT MISSING ALLELES PER SITE
complete=genotype.count_missing(axis=1)

In [19]:
# TURN INTO BOOLEAN
complete=(complete==0)
complete

dask.array<eq, shape=(37199402,), dtype=bool, chunksize=(279737,), chunktype=numpy.ndarray>

In [32]:
# NUMBER OF SITES WITH COMPLETE INFO (OUT OF 37.199M)
da.sum(complete).compute()

31476735

##### 2.3 Subset individuals
In fact I have alrady subsetted the individuals when running snp_calls() with sample_sets, so no additional steps need to be done. But if you have not already done that, or your previous filters need to be applied on the global population, then you now need to create a boolean vector (with length equals the global sample size of 3081) to indicate the mosquito(es) you need. Or, equivalently, to create a vector indicating which samples (1st, 22nd, 30th, etc) you are selecting. For the latter case, your vector length should equal the number of mosquitoes you are selecting, and you should use take() instead of compress(). 

#### 3. Compress
I don't like the term compress because it is not exactly what it does. It is not the case that you compress a file into zip or rar then you can extract it and retrieve the same information. Compress here means to get rid of some samples and SNPs, or to draw a subset. It's more like compressing lossless music into mp3, say. 

As you can compress samples and/or sites there are two dimensions you need to work towards. axis=0 is sites and axis=1 is mosquito samples. Besides, if you are getting rid of some sites (which is very likely), then do not forget to compress the position vector as well. 

What we want is fixed_biallelic AND intergenic AND complete sites. We can pick this up with by multiplying these boolean vectors together. 

In [20]:
# MULTIPLY BOOLEAN VECTORS. AND I DON'T KNOW HOW IT AUTOMATICALLY BECOMES A DASK ARRAY
# CAN WE DO THREE MULTIPLICATIONS IN ONE STEP?
#what_we_want=np.multiply(biallelic, intergenic)
what_we_want=np.multiply(fixed_biallelic, intergenic)
what_we_want=np.multiply(what_we_want, complete)
what_we_want

dask.array<multiply, shape=(37199402,), dtype=bool, chunksize=(279737,), chunktype=numpy.ndarray>

In [21]:
what_we_want[0:20].compute()

array([False, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

Normally I'd like to calculate the sum of the boolean vector what_we_want to see how many 1's there are. But we will get the answer anyway in the next step when we compress the genotype. 

In [22]:
# COMPRESS GENOTYPE. NEED FAST INTERNET
subset_genotype=genotype.compress(what_we_want, axis=0)
subset_genotype
# .take() IF WE HAVE THE COLUMN INDEX WE NEED

<GenotypeDaskArray shape=(14463262, 283, 2) dtype=int8>

In [23]:
# COMPRESS POSITION. NEED FAST INTERNET
# REMEMBER POS ALREADY DEFINED
#POS=allel.SortedIndex(ds_snps["variant_position"])
subset_POS=POS.compress(what_we_want)
subset_POS

# ANOTHER WAY, NOT RECOMMENDED
#POS=POS.compute()
#subset_POS=POS[biallelic]

0,1,2,3,4,...,14463257,14463258,14463259,14463260,14463261
236,239,240,241,242,...,53196499,53196500,53196502,53196504,53196522


In [ ]:
# ANOTHER WAY TO DRAW SUBSET, BUT NOT RECOMMENDED
#pos1=ds_snps["variant_position"].data
#pos1=pos1.compute()
#subset_pos1=pos1[what_we_want]
#subset_pos1

1) Let us look at the numbers. There are 37.199M passed sites on 3R, and there are 14.46M sites which are intergenic + complete + have two variants or less. Note that last time we extracted 4.6M intergenic + complete + biallelic SNPs. These numbers are similar to what Jon prepared in Hui et al. 2021 using the pre-release of phase 3 data (~3.3M SNPs). 

2) The compress() step is slow. I have fibre broadband at home but it is still a lot slower than I expected. For example, compressing the position array (~140MB) invokes heavy internet traffic for a long duration. And it takes up 6GB of memory at peak. I assume there are ways to compress multiple chunks of dask array in parallel, but I don't know how. It took me about 30-45 minutes at home to compress both genotype and position. Is that normal?

3) Another performance issue. Again tt is slow and memory hungry (4GB) to calculate the sum (via da.sum()) of boolean vectors. I don't understand why, given the small data size (37M sites = 37MB size of the boolean vector. I assume boolean is stored as 1-byte instead of 1-bit for memory alignment). 

4) To compress individuals you need to run the compress() again on another dimension (axis=1). 

5) You should always check the dimensions of subset_genotype and subset_POS, and the boolean vectors you created. 

#### 4. Write to hdf5
It is always my interest to export the subsets to the R-supported hdf5 file format. I understand there is a workgroup (whose members include Alistair) wanting to create a zarr API for R, but it very much is still an on-going project. 

Note: I confirm that it is not necessary to rechunk subset_genotype as writing to hdf5 will involve rechunking anyway (rechunking is slow again!). It is however essential to turn subset_POS into a dask array. I also want to return sample_ID in my subset for clarity, but string vectors like this need to be recoded as utf-8. 

In [24]:
# RECHUNK subset_genotype (NOT NEEDED NOW)
#subset_genotype.rechunk(chunks=(279737, 50, 2))
#subset_genotype

In [25]:
# TURN subset_POS (numpy array?) INTO A DASK ARRAY
subset_POS=da.from_array(subset_POS, chunks=279373)
subset_POS

dask.array<array, shape=(14463262,), dtype=int32, chunksize=(279373,), chunktype=numpy.ndarray>

In [26]:
# I'D LIKE TO HAVE THE sample_id AS WELL (FROM METADATA, OR BF_metadata WE CREATED)
# NEED TO CODE THE STRINGS IN utf-8 OTHERWISE IT WILL UPSET hdf5
BF_sample_id=BF_metadata["sample_id"]
BF_sample_id=list([bytes(bf, "utf-8") for bf in BF_sample_id])
BF_sample_id

[b'AB0085-Cx',
 b'AB0086-Cx',
 b'AB0087-C',
 b'AB0088-C',
 b'AB0089-Cx',
 b'AB0090-C',
 b'AB0091-C',
 b'AB0092-C',
 b'AB0094-Cx',
 b'AB0095-Cx',
 b'AB0096-C',
 b'AB0097-Cx',
 b'AB0098-Cx',
 b'AB0099-Cx',
 b'AB0100-C',
 b'AB0101-C',
 b'AB0103-C',
 b'AB0104-Cx',
 b'AB0109-C',
 b'AB0110-C',
 b'AB0111-C',
 b'AB0112-C',
 b'AB0113-C',
 b'AB0114-C',
 b'AB0115-C',
 b'AB0116-C',
 b'AB0117-C',
 b'AB0118-C',
 b'AB0119-C',
 b'AB0121-C',
 b'AB0122-C',
 b'AB0123-C',
 b'AB0124-C',
 b'AB0126-Cx',
 b'AB0127-C',
 b'AB0128-C',
 b'AB0129-C',
 b'AB0130-Cx',
 b'AB0131-Cx',
 b'AB0132-C',
 b'AB0133-C',
 b'AB0134-C',
 b'AB0135-C',
 b'AB0136-Cx',
 b'AB0137-Cx',
 b'AB0138-Cx',
 b'AB0139-C',
 b'AB0140-C',
 b'AB0142-C',
 b'AB0143-Cx',
 b'AB0144-C',
 b'AB0145-C',
 b'AB0146-Cx',
 b'AB0147-C',
 b'AB0148-Cx',
 b'AB0150-Cx',
 b'AB0151-Cx',
 b'AB0153-C',
 b'AB0155-Cx',
 b'AB0157-Cx',
 b'AB0158-Cx',
 b'AB0159-C',
 b'AB0160-Cx',
 b'AB0161-C',
 b'AB0162-C',
 b'AB0164-C',
 b'AB0165-C',
 b'AB0166-C',
 b'AB0167-C',
 b'AB0169-

I don't understand why rechunking subset_genotype invokes internet traffic as the dataset should already exist in my local ram/storage. 

##### first method: via h5py
I think this is the more elegant solution that puts multiple datasets in one hdf5. I don't know how you can save files when using google cloud but I am sure there is a way (with google drive I guess). It appears that I need to specify chunks=() as a tuple (even for 1D array like subset_POS), and even if I had done the chunking before. 

In [27]:
# WRITE TO hdf5
# LARGE FILE, WRITE TO HARD DRIVE (NOT ONE DRIVE)
# create_dataset() ONE BY ONE, ALL TO THE SAME hdf5
import h5py
f=h5py.File('G:/3R_BFAB_pass_fixedbiallelic_intergenic_complete.hdf5', 'a')
f.create_dataset(name='/POS', data=subset_POS, chunks=(279373,))
f.create_dataset(name='/genotype',data=subset_genotype, chunks=(279373, 50, 1))
f.create_dataset(name='/sample_id', data=BF_sample_id)

<HDF5 dataset "sample_id": shape (283,), type "|S9">

In [28]:
# REMEMBER TO CLOSE FILE CONNECTION
f
f.close()

Again writing hdf5 seems to invoke internet traffic (why?). 

##### second method: one array per hdf5 
You may directly use .to_hdf5() with subset_POS and subset_genotype without importing h5py. Then you have separate hdf5 files. 

In [14]:
# IF YOU WANT TO WRITE ONE ARRAY TO ONE hdf5
subset_POS.to_hdf5('test_pos.hdf5', '/POS')
subset_genotype.to_hdf5("test_genotype.hdf5", '/genotype')

##### END
It is never the best version but at least it does the job. The same script can be used to select other populations, chromosome arms and segments, types of SNPs etc. 

There are further subsetting needs to be done with the hdf5 (such as removing the arabiensis and missing alleles, recoding the genotypes, filtering maf, splitting by year etc.) but I can perform them in R as a free person :-)